In [1]:
import os
execfile(os.path.join(os.environ["SPARK_HOME"], 'python/pyspark/shell.py'))

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 1.6.0
      /_/

Using Python version 2.7.12 (default, Nov 19 2016 06:48:10)
SparkContext available as sc, HiveContext available as sqlContext.


In [45]:
from pyspark.mllib.classification import LogisticRegressionWithLBFGS, LogisticRegressionModel
from pyspark.mllib.regression import LabeledPoint
import numpy

In [60]:
# Function to convert .csv files into 'LabeledPoint' format
def parsePoint(line):
    values = [float(x.strip()) for x in line.split(',')]
    #biao qian cong 0 kaishi
    if values[-1]==1:
        values[-1]=0
    else:
        values[-1]=1
    return LabeledPoint(values[-1],values[:24])

In [61]:

# Load .csv data
data = sc.textFile("hdfs:///user/german_numeric.csv")
train, test = data.randomSplit([0.8, 0.2], seed=12345)
print test.count(),train.count()

182 818


In [72]:
# Convert the data to LabeledPoint format
train_parsed = train.map(parsePoint)
test_parsed = test.map(parsePoint)
print train_parsed.first()

(0.0,[1.0,6.0,4.0,12.0,5.0,5.0,3.0,4.0,1.0,67.0,3.0,2.0,1.0,2.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0])


In [64]:
# Build the model
model = LogisticRegressionWithLBFGS.train(train_parsed, iterations=10,numClasses=2)

# Evaluating the model on training data
labelsAndPreds = train_parsed.map(lambda p: (p.label, model.predict(p.features)))
trainErr = labelsAndPreds.filter(lambda (v, p): v != p).count() / float(train_parsed.count())
print("Training Error = " + str(trainErr))


Training Error = 0.213936430318


In [69]:
predictions = model.predict(test_parsed.map(lambda x: x.features))
print predictions.collect()

[1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0]
